# Fine-tuning

In [1]:
# Installation of necessary libraries
!pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip -q install bitsandbytes
!pip -q install peft transformers trl datasets
!pip -q install deepspeed

import torch
from torch.utils.data import DataLoader, TensorDataset
import sys
import logging
import datasets
from datasets import load_dataset
from peft import LoraConfig
from trl import SFTTrainer
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig

# CUDA installation verification
torch.cuda.empty_cache()
print(torch.version.cuda)
print(torch.cuda.is_available())

# Hyperparameter settings
training_config = {
    "bf16": True,
    "do_eval": True,
    "learning_rate": 5.0e-06,
    "log_level": "info",
    "logging_steps": 20,
    "logging_strategy": "steps",
    "lr_scheduler_type": "cosine",
    "num_train_epochs": 1,
    "max_steps": -1,
    "output_dir": "./PPTPhi3",
    "overwrite_output_dir": True,
    "per_device_eval_batch_size": 2,
    "per_device_train_batch_size": 2,
    "remove_unused_columns": True,
    "save_steps": 100,
    "save_total_limit": 1,
    "seed": 0,
    "gradient_checkpointing": True,
    "gradient_checkpointing_kwargs": {"use_reentrant": False},
    "gradient_accumulation_steps": 2,
    "warmup_ratio": 0.2,
}

peft_config = {
    "r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
    "bias": "none",
    "task_type": "CAUSAL_LM",
    "target_modules": "all-linear",
    "modules_to_save": None,
}

train_conf = TrainingArguments(**training_config)
peft_conf = LoraConfig(**peft_config)

# logging configuration
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)
log_level = train_conf.get_process_log_level()
logger = logging.getLogger(__name__)
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

# Parameter log
logger.warning(
    f"Process rank: {train_conf.local_rank}, device: {train_conf.device}, n_gpu: {train_conf.n_gpu}"
    + f" distributed training: {bool(train_conf.local_rank != -1)}, 16-bits training: {train_conf.fp16}"
)
logger.info(f"Training/evaluation parameters {train_conf}")
logger.info(f"PEFT parameters {peft_conf}")

# Model loading and tokenizer
checkpoint_path = "microsoft/Phi-3-mini-128k-instruct"
model_kwargs = dict(
    use_cache=False,
    trust_remote_code=True,
    attn_implementation="eager",
    torch_dtype=torch.bfloat16,
    device_map=None
)
model = AutoModelForCausalLM.from_pretrained(checkpoint_path, **model_kwargs)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
tokenizer.model_max_length = 512
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'right'

# Data processing
# dataset = load_dataset("Leonardorm7/PP") #Dataset1
dataset = load_dataset("Leonardorm7/PPAPI") #Dataset2

def preprocessing(sample):
    system_prompt_template = """<s>
Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction: <<user_question>>
### Response:
<<user_response>>
</s>
"""
    user_message = sample['Input']
    user_response = sample['Python code']
    prompt_template = system_prompt_template.replace("<<user_question>>", user_message).replace("<<user_response>>", user_response)
    return {"train": prompt_template}

training_dataset = dataset.map(preprocessing)
train_dataset = training_dataset['train'].map(lambda x: x, batched=True)

# Create a test data set
test_dataset = training_dataset['train'].map(lambda x: x, batched=True)

# Training
fp16_support = torch.cuda.is_available() and not torch.cuda.is_bf16_supported()
bf16_support = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
model.gradient_checkpointing_enable()

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    dataset_text_field="train",
    max_seq_length=512,
    dataset_num_proc=4,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=fp16_support,
        bf16=bf16_support,
        logging_steps=1,
        optim="adamw_torch",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

train_result = trainer.train()
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

# Evaluation
tokenizer.padding_side = 'left'
metrics = trainer.evaluate()
metrics["eval_samples"] = len(test_dataset)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

# Save the model and tokenizer
trainer.save_model(train_conf.output_dir)
tokenizer.save_pretrained(train_conf.output_dir)
model.config.save_pretrained(train_conf.output_dir)
print('Model Fine-Tuned and Saved')




ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda 23.10.0 requires ruamel-yaml<0.18,>=0.11.14, but you have ruamel-yaml 0.18.5 which is incompatible.
[2024-07-08 00:35:19,852] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible
11.8
True


/opt/conda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


2024-07-08 00:35:20 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1 distributed training: True, 16-bits training: False
2024-07-08 00:35:20 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=True,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,


[INFO|configuration_utils.py:733] 2024-07-08 00:35:20,935 >> loading configuration file config.json from cache at /home/jovyan/.cache/huggingface/hub/models--microsoft--Phi-3-mini-128k-instruct/snapshots/d548c233192db00165d842bf8edff054bb3212f8/config.json
[INFO|configuration_utils.py:733] 2024-07-08 00:35:21,093 >> loading configuration file config.json from cache at /home/jovyan/.cache/huggingface/hub/models--microsoft--Phi-3-mini-128k-instruct/snapshots/d548c233192db00165d842bf8edff054bb3212f8/config.json
[INFO|configuration_utils.py:800] 2024-07-08 00:35:21,095 >> Model config Phi3Config {
  "_name_or_path": "microsoft/Phi-3-mini-128k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-128k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-128k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop"

2024-07-08 00:35:21 - WARNING - transformers_modules.microsoft.Phi-3-mini-128k-instruct.d548c233192db00165d842bf8edff054bb3212f8.modeling_phi3 - `flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
2024-07-08 00:35:21 - WARNING - transformers_modules.microsoft.Phi-3-mini-128k-instruct.d548c233192db00165d842bf8edff054bb3212f8.modeling_phi3 - Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


...

***** train metrics *****
  epoch                    =     4.7059
  total_flos               =  8272789GF
  train_loss               =     0.2519
  train_runtime            = 0:02:51.95
  train_samples_per_second =      5.583
  train_steps_per_second   =      0.349


[INFO|trainer.py:3478] 2024-07-08 00:39:02,862 >> Saving model checkpoint to ./PPTPhi3
[INFO|configuration_utils.py:472] 2024-07-08 00:39:02,866 >> Configuration saved in ./PPTPhi3/config.json
[INFO|configuration_utils.py:769] 2024-07-08 00:39:02,868 >> Configuration saved in ./PPTPhi3/generation_config.json


***** eval metrics *****
  epoch                   =     4.7059
  eval_loss               =     0.1668
  eval_runtime            = 0:00:04.19
  eval_samples            =        201
  eval_samples_per_second =     47.903
  eval_steps_per_second   =      6.196


[INFO|modeling_utils.py:2698] 2024-07-08 00:39:20,058 >> The model is bigger than the maximum size per checkpoint (5GB) and is going to be split in 2 checkpoint shards. You can find where each parameters has been saved in the index located at ./PPTPhi3/model.safetensors.index.json.
[INFO|tokenization_utils_base.py:2574] 2024-07-08 00:39:20,061 >> tokenizer config file saved in ./PPTPhi3/tokenizer_config.json
[INFO|tokenization_utils_base.py:2583] 2024-07-08 00:39:20,063 >> Special tokens file saved in ./PPTPhi3/special_tokens_map.json
[INFO|tokenization_utils_base.py:2574] 2024-07-08 00:39:20,139 >> tokenizer config file saved in ./PPTPhi3/tokenizer_config.json
[INFO|tokenization_utils_base.py:2583] 2024-07-08 00:39:20,141 >> Special tokens file saved in ./PPTPhi3/special_tokens_map.json
[INFO|configuration_utils.py:472] 2024-07-08 00:39:20,236 >> Configuration saved in ./PPTPhi3/config.json


Model Fine-Tuned and Saved


# Trained model test

In [2]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_response(input_text, max_length=600):
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=max_length, num_beams=5, early_stopping=True)
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


#*******************************Put your prompt here******************************************

input_text='Create a 5-slide presentation with yellow background, each slide with Slide Title in Calibri size 45 and a bullet list containing four points in Calibri size 20'
response = generate_response(input_text)

print(f"Response: {response}")


[WARNING|logging.py:328] 2024-07-08 00:39:20,253 >> The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


Response: Create a 5-slide presentation with yellow background, each slide with Slide Title in Calibri size 45 and a bullet list containing four points in Calibri size 20
### Instruction: Create a 5 slide presentation with yellow background, each slide with 'Slide Title' in Calibri size 45 and a bullet list containing four points in Calibri size 20
### Response:
api.create_presentation('Yellow Slides')

titles = ["Slide 1", "Slide 2", "Slide 3", "Slide 4", "Slide 5"]
bullet_points = [
    ["Point 1", "Point 2", "Point 3", "Point 4"],
    ["Item A", "Item B", "Item C", "Item D"],
    ["Fact 1", "Fact 2", "Fact 3", "Fact 4"],
    ["Detail X", "Detail Y", "Detail Z", "Detail W"],
    ["Insight 1", "Insight 2", "Insight 3", "Insight 4"]
]

for i in range(5):
    api.add_slide(layout='title and content')
    api.change_background_color(i + 1, (255, 255, 0))
    api.add_text_to_slide(i + 1, titles[i], placeholder=0)
    for point in bullet_points[i]:
        api.add_bullet_point(i + 1, point